In [1]:
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

N = config["rnn"]["simulation"]["N"]
timesteps = config["rnn"]["simulation"]["timesteps"]
dt = config["rnn"]["simulation"]["dt"]
alpha = config["rnn"]["simulation"]["alpha"]
beta = config["rnn"]["simulation"]["beta"]

seed = config["rnn"]["model"]["seed"]
train_size = config["rnn"]["model"]["train_size"]
val_size = config["rnn"]["model"]["val_size"]
test_size = config["rnn"]["model"]["test_size"]

loss = config["rnn"]["training"]["loss"]
save = config["rnn"]["data"]["save"]
load = config["rnn"]["data"]["load"]

optimization_steps = config["gridsearch"]["optimization_steps"]
hyperparameters = config["gridsearch"]["hyperparameters"]

In [2]:
import random
random.seed(seed)

# Create an instance of the Simulation class
import numpy as np
np.random.seed(seed)
import functions
import os

# Specify the folder path
folder_path = r"D:\\School\\Magistrska\\data"

q_i = np.sin(np.linspace(0, np.pi*2, N))
p_i = np.zeros(N)

sim = functions.Simulation(q_i, p_i, N, timesteps, dt, alpha, beta)

# Check if the folder is empty
if len(os.listdir(folder_path)) == 0:
    print("No saved data, running simulation.")

    qs, ps = sim.integrate(q_i, p_i)
    
else:
    print("Found some saved data, skipping simulation.")
    
    qs = np.load(folder_path+r"\data_qs_1.npy")
    ps = np.load(folder_path+r"\data_ps_1.npy")
    
qs = qs[:, 1:-1]
ps = ps[:, 1:-1]

Found some saved data, skipping simulation.


In [ ]:
from sklearn.model_selection import ParameterGrid

all_parameter_comb = ParameterGrid(hyperparameters)
print(len(all_parameter_comb))

In [ ]:
# Randomize the order of the gridsearch
# random.seed(seed)

all_parameter_comb = random.sample(list(all_parameter_comb), optimization_steps)

In [3]:
import tensorflow as tf
tf.random.set_seed(seed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, GRU
# Enable multi-threading and multiprocessing
tf.config.threading.set_inter_op_parallelism_threads(2)
tf.config.threading.set_intra_op_parallelism_threads(2)

def build_model(learning_rate, hidden_units_per_layer, dropout, window_size):
    
    initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=seed)

    model = Sequential()
    model.add(InputLayer((window_size, N-2)))
    
    model.add(GRU(hidden_units_per_layer, activation="tanh", kernel_initializer=initializer))
     
    if dropout != 0:
        model.add(Dropout(dropout, seed=seed))
        
    model.add(Dense(N-2, kernel_initializer=initializer))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(loss="mse", optimizer=optimizer)

    return model

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json

best_mse = np.inf
best_params = {}
best_model = None
best_pred = []

iteration = 1

for params in all_parameter_comb:
    
    X, y = functions.make_sequences(functions.preprocess(qs), params["window_size"])

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_size/(1-val_size), shuffle=False)

    from tensorflow.keras.callbacks import ModelCheckpoint

    # Create the ModelCheckpoint callback
    model_checkpoint_callback = ModelCheckpoint(filepath=(r'D:\\School\\Magistrska\\best_model_RNN\\gridsearch_temp.h5'), monitor='val_loss', save_best_only=True, mode='min', verbose=0)

    model = build_model(params["learning_rate"], params["hidden_units_per_layer"], params["dropout"], params["window_size"])

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=params["epochs"], batch_size=params["batch_size"], callbacks=[model_checkpoint_callback], verbose = 0)
    
    model = tf.keras.models.load_model(r'D:\\School\\Magistrska\\best_model_RNN\\gridsearch_temp.h5')
    
    test_pred = np.copy(X_test[0])
    for k in range(len(X_test)):
        pred = model.predict(test_pred[np.newaxis, -params["window_size"]:], verbose=0)
        test_pred = np.concatenate((test_pred, pred), axis = 0)
        
    test_pred = test_pred[params["window_size"]:]

    mse = mean_squared_error(functions.postprocess(y_test), functions.postprocess(test_pred))

    if mse < best_mse:
        best_mse = mse
        best_params = params
        best_model = model
        best_pred = test_pred

        print("Best Parameters:", best_params)
        print("Best mse:", best_mse)
        
        plt.plot(functions.postprocess(y_test)[:, 2], color="tab:blue")
        plt.plot(functions.postprocess(test_pred)[:, 2], color="tab:orange")
        plt.ylim(-1.2, 1.2)
        plt.show()
        
        # Save on each step in case of crash        
        best_model.save(r'D:\\School\\Magistrska\\best_model_RNN\\gridsearch_1.h5')
        
        np.save(r'D:\\School\\Magistrska\\best_model_RNN\\best_pred_1.npy', best_pred)

        with open(r'D:\\School\\Magistrska\\best_model_RNN\\best_params_1.json', 'w') as config_file:
            json.dump(best_params, config_file)
        
    print("\n Completed: ", iteration, "/", len(all_parameter_comb))
    print("-------------------------------------------------------------")

    iteration += 1

In [17]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model(r"D:\School\Magistrska\best_model_RNN\gridsearch_1.h5")

# Convert the Keras model to a frozen graph
frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
    tf.compat.v1.Session(),
    tf.compat.v1.Session().graph_def,
    [model.output.op.name]
)

# Create a TensorFlow session and import the frozen graph
with tf.compat.v1.Session() as sess:
    tf.compat.v1.graph_util.import_graph_def(frozen_graph)

    # Access the input and output nodes of the graph
    input_node = sess.graph.get_tensor_by_name('input_node_name:0')
    output_node = sess.graph.get_tensor_by_name('output_node_name:0')

    # Perform inference on new data
    input_data = ...  # Prepare your input data
    predictions = sess.run(output_node, feed_dict={input_node: input_data})

# Process the predictions as needed


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


AssertionError: dense_120/BiasAdd_6 is not in graph

In [ ]:
with open(r"D:\\School\\Magistrska\\best_model_RNN\\best_params_1.json", 'r') as config_file:
    best_params = json.load(config_file)
    
best_pred = np.load(r'D:\\School\\Magistrska\\best_model_RNN\\best_pred_1.npy')

In [10]:
X, y = functions.make_sequences(functions.preprocess(qs), best_params["window_size"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_size/(1-val_size), shuffle=False)

test_pred = np.copy(X_test[0])
for k in range(len(X_test)):
    pred = optimal_model.predict(test_pred[np.newaxis, -best_params["window_size"]:], verbose=0)
    test_pred = np.concatenate((test_pred, pred), axis=0)
    
test_pred = test_pred[best_params["window_size"]:]

mse = mean_squared_error(functions.postprocess(y_test), functions.postprocess(test_pred))

print("Optimal Parameters:", best_params)
print("Optimal mse:", mse)

NameError: name 'best_params' is not defined

In [ ]:
for particles in range(N-2):
    plt.plot(functions.preprocess(y_test)[:, particles], color="tab:blue")
    plt.plot(functions.postprocess(test_pred)[:, particles], color="tab:orange")
    # plt.plot(postprocess(best_pred)[:, particles], color="tab:red")
    # plt.ylim(-1.2, 1.2)
    plt.savefig("plot_particle_#"+str(particles)+".pdf", format='pdf')
    plt.show()